In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display
import scipy.misc
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
#import essentia, essentia.standard as ess

In [2]:
def chargen(char=20):
    print char
    char_gen = []
    i=0
    genre={}
    cont=0
    for folder in os.listdir("./genres"): #iterates all folders    

        folder="./genres/"+folder
        if not folder in genre.keys(): #creates a dictionary genre->number
            genre[folder]=i
            i+=1

        for filename in os.listdir(folder): #iterates all files in the folder
            filename = folder+"/"+filename

            if filename.endswith((".au",".mp3",".wav",".aiff")):
                audio, fs = librosa.load(filename, duration=29.0, mono=True, sr=22050) #loads the first 30 seconds of the song
                mfcc = librosa.feature.mfcc(audio, sr=fs, n_mfcc=char) #calculates MFCC of the song
                #The size of the matrix will be number_of_MFCCs*1292, which is a feature every 0.02s
                #By default, it's 20*1291

                """The first comlumn gives useless information, so we remove it
                Works as offset"""
                mfcc = np.delete(mfcc, 0, axis=1)
                #mfcc = np.ndarray.tolist(mfcc)

                char_gen.append((mfcc,genre[folder])) #adds the array to a list
                
                if cont%10==0:
                    print cont%100,
                
                cont+=1
        print 
                    
    return np.array(char_gen)

In [3]:
def char1():
    char_gen=[]
    i=0
    genre={}
    cont=0
    for folder in os.listdir("./genres"): #iterates all folders    

        folder="./genres/"+folder
        if not folder in genre.keys(): #creates a dictionary genre->number
            genre[folder]=i
            i+=1

        for filename in os.listdir(folder): #iterates all files in the folder
            filename = folder+"/"+filename

            if filename.endswith((".au",".mp3",".wav",".aiff")):
                audio, fs = librosa.load(filename, duration=29.0, mono=True, sr=22050) #loads the first 30 seconds of the song
                mfcc = librosa.feature.mfcc(audio, sr=fs) #calculates MFCC of the song
                #The size of the matrix will be number_of_MFCCs*1292, which is a feature every 0.02s
                #By default, it's 20*1291

                """The first comlumn gives useless information, so we remove it
                Works as offset"""
                mfcc = np.delete(mfcc, 0, axis=1)
                                
                #librosa.display.specshow(mfcc, x_axis='time')
                #plt.show()

                mfcc=np.transpose(mfcc)
                mfcc=mfcc.flatten() #converts the matrix into a 1D array

                char_gen.append((mfcc,genre[folder])) #adds the array to a list
                
                if cont%10==0:
                    print cont%100,
                
                cont+=1
                '''if cont>=10:
                    break'''
        print 
                    
    return np.array(char_gen)

In [4]:
def char2():
    char_gen=[]
    i=0
    genre={}
    for folder in os.listdir("./genres"): #iterates all folders    
        cont=0

        folder="./genres/"+folder
        if not folder in genre.keys(): #creates a dictionary genre->number
            genre[folder]=i
            i+=1

        for filename in os.listdir(folder): #iterates all files in the folder
            filename = folder+"/"+filename

            if filename.endswith((".au",".mp3",".wav",".aiff")):
                audio, fs = librosa.load(filename, duration=30.0, mono=True, sr=22050) #loads the first 30 seconds of the song
                mfcc = librosa.feature.mfcc(audio, sr=fs, n_fft=1024, hop_length=30*22050) #calculates MFCC of the song
                #The size of the matrix will be number_of_MFCCs*1292, which is a feature every 0.02s
                #By default, it's 20*1922

                """The first comlumn gives useless information, so we remove it
                Works as offset"""
                mfcc = np.delete(mfcc, 0, axis=1)
                
                #librosa.display.specshow(mfcc, x_axis='time')
                #plt.show()
                
                mfcc=mfcc.flatten() #converts the matrix into a 1D array

                char_gen.append((mfcc,genre[folder])) #adds the array to a list

                cont+=1
                if cont>=1:
                    break
                
    return char_gen

In [17]:
def char3(char=5):
    print char
    char_gen=[]
    maxs=[]
    mins=[]
    i=0
    genre={}
    cont=0
    scaler = StandardScaler()
    for folder in os.listdir("./genres"): #iterates all folders    

        folder="./genres/"+folder
        if not folder in genre.keys(): #creates a dictionary genre->number
            genre[folder]=i
            i+=1

        print str(cont/100)+":",    
            
        for filename in os.listdir(folder): #iterates all files in the folder
            filename = folder+"/"+filename

            if filename.endswith((".au",".mp3",".wav",".aiff")):
                audio, fs = librosa.load(filename, duration=29.0, mono=True, sr=22050) #loads the first 30 seconds of the song
                mfcc = librosa.feature.mfcc(audio, sr=fs, n_mfcc=char) #calculates MFCC of the song
                #The size of the matrix will be number_of_MFCCs*1292, which is a feature every 0.02s
                #By default, it's 20*1291

                """The first comlumn gives useless information, so we remove it
                Works as offset"""
                mfcc = np.delete(mfcc, 0, axis=1)
                
                #mfcc = scaler.fit_transform(mfcc)
                                
                maxs.append(np.amax(mfcc))
                mins.append(np.amin(mfcc))
                
                #mfcc=np.transpose(mfcc)
                #mfcc=mfcc.flatten() #converts the matrix into a 1D array

                char_gen.append((mfcc,genre[folder])) #adds the array to a list
                
                if cont%10==0:
                    print cont%100,
                
                cont+=1
        print 
    step = (max(maxs) - min(mins))/100
    return np.array(char_gen), step

In [6]:
def histogram(step, char, amount):
    histogram = np.zeros(amount)
    for c in char:
        histogram[int(c/step)] += 1.
    histogram = histogram/len(char)
    histogram = np.ndarray.tolist(histogram)
    return histogram

In [7]:
def reshape_pca(char):
    mfcc = np.array(char[:,0 ])
    mfcc = np.concatenate(mfcc).reshape(1000,24960)
    pca=PCA(n_components = 100, svd_solver='full')
    pca.fit(mfcc)
    mfcc=pca.transform(mfcc)
    return mfcc

In [23]:
def train_test(mfcc, char):
    train_mfcc, test_mfcc=[],[]
    train_genre, test_genre=[], []

    for i in range(1000):
        if i%10==0:
            test_mfcc.append(mfcc[i])
            test_genre.append(char[i][1])
        else:
            train_mfcc.append(mfcc[i])
            train_genre.append(char[i][1])

    train_mfcc = np.array(train_mfcc)
    test_mfcc = np.array(test_mfcc)
    train_genre = np.array(train_genre)
    test_genre = np.array(test_genre)
    return train_mfcc, train_genre, test_mfcc, test_genre

In [24]:
def classify(train_mfcc, train_genre, test_mfcc, test_genre):
    success=0
    clf=SVC(C=0.0001)
    clf.fit(train_mfcc,train_genre)
    for i in range(100):
        test=test_mfcc[i].reshape(1, -1)
        pred = clf.predict(test)
        if pred[0]==test_genre[i]:
            success+=1
    return success

In [10]:
def all_hist(char, step, amount):
    hist=[]
    temp_char=[]
    for c in char:
        temp = histogram(step, c[0], amount)
        hist.append((temp, c[1]))
        temp_char.append(temp)
    return hist, temp_char


In [11]:
def more_hist(char, step, amount):
    hist = []
    temp_char=[]
    for c in char:
        tmp = c[0]
        temp = []
        for v in tmp:
            temp+=histogram(step, v, amount)
        temp_char.append(temp)
        hist.append((temp, c[1]))
    return hist, temp_char    

In [12]:
def mode1():
    char = char1()
    mfcc = reshape_pca(char)
    train_mfcc, train_genre, test_mfcc, test_genre = train_test(mfcc, char)
    classify(train_mfcc, train_genre, test_mfcc, test_genre)

In [13]:
def mode2():
    char = char2()
    mfcc = reshape_pca(char)
    train_mfcc, train_genre, test_mfcc, test_genre = train_test(mfcc, char)
    classify(train_mfcc, train_genre, test_mfcc, test_genre)

In [14]:
def mode3(char, step, amount):
    hist, char = all_hist(char, step, amount)
    train_mfcc, train_genre, test_mfcc, test_genre = train_test(char, hist)
    return classify(train_mfcc, train_genre, test_mfcc, test_genre)

In [15]:
def mode4(char, step, amount):
    hist, char = more_hist(char, step, amount)
    train_mfcc, train_genre, test_mfcc, test_genre = train_test(char, hist)
    return classify(train_mfcc, train_genre, test_mfcc, test_genre)

In [36]:
for i in range(5,21,5):
    char_gen = chargen(i)
    filename = "values"+str(i)
    np.save(filename, char_gen)

 5
0 10 20 30 40 50 60 70 80 90
0 10 20 30 40 50 60 70 80 90
0 10 20 30 40 50 60 70 80 90
0 10 20 30 40 50 60 70 80 90
0 10 20 30 40 50 60 70 80 90
0 10 20 30 40 50 60 70 80 90
0 10 20 30 40 50 60 70 80 90
0 10 20 30 40 50 60 70 80 90
0 10 20 30 40 50 60 70 80 90
0 10 20 30 40 50 60 70 80 90
10
0 10 20 30 40 50 60 70 80 90
0 10 20 30 40 50 60 70 80 90
0 10 20 30 40 50 60 70 80 90
0 10 20 30 40 50 60 70 80 90
0 10 20 30 40 50 60 70 80 90
0 10 20 30 40 50 60 70 80 90
0 10 20 30 40 50 60 70 80 90
0 10 20 30 40 50 60 70 80 90
0 10 20 30 40 50 60 70 80 90
0 10 20 30 40 50 60 70 80 90
15
0 10 20 30 40 50 60 70 80 90
0 10 20 30 40 50 60 70 80 90
0 10 20 30 40 50 60 70 80 90
0 10 20 30 40 50 60 70 80 90
0 10 20 30 40 50 60 70 80 90
0 10 20 30 40 50 60 70 80 90
0 10 20 30 40 50 60 70 80 90
0 10 20 30 40 50 60 70 80 90
0 10 20 30 40 50 60 70 80 90
0 10 20 30 40 50 60 70 80 90
20
0 10 20 30 40 50 60 70 80 90
0 10 20 30 40 50 60 70 80 90
0 10 20 30 40 50 60 70 80 90
0 10 20 30 40 50 60 70 80 90
0 

In [49]:
for i in range(5,6,5):
    filename = "values"+str(i)+".npy"
    char = np.load(filename)
    #print char.shape, char[0].shape, char[0][0].shape, char[0][0][0].shape
    #step = (np.amax(np.amax(np.amax(char))) - np.amin(np.amin(np.amin(char))))/100

In [74]:
char, step = char3(5)

0: 0 10 20 30 40 50 60 70 80 90
1: 0 10 20 30 40 50 60 70 80 90
2: 0 10 20 30 40 50 60 70 80 90
3: 0 10 20 30 40 50 60 70 80 90
4: 0 10 20 30 40 50 60 70 80 90
5: 0 10 20 30 40 50 60 70 80 90
6: 0 10 20 30 40 50 60 70 80 90
7: 0 10 20 30 40 50 60 70 80 90
8: 0 10 20 30 40 50 60 70 80 90
9: 0 10 20 30 40 50 60 70 80 90


In [54]:
hist, charco = more_hist(char,(step*100)/50, 50)

([array([0.03044872, 0.00080128, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.00240385, 0.00641026, 0.02083333,
       0.03125   , 0.06410256, 0.11939103, 0.11378205, 0.15865385,
       0.14663462, 0.10416667, 0.08333333, 0.0713141 , 0.04647436]), array([0.        , 0.        , 0.00160256, 0.00961538, 0.04407051,
       0.24759615, 0.43589744, 0.21794872, 0.04326923, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0. 

In [57]:
results=[]
for i in range(1,100):
    if i%10==0: print i,
    results.append((i, mode3(char, (step*100)/i, amount=i)))

print
results.sort(key=lambda x:x[1], reverse=True)
print results[0]

TypeError: only size-1 arrays can be converted to Python scalars

In [96]:
results=[]
for i in range(5,100):
    if i%10==0: print i,
    results.append((i, mode4(char, (step*100)/i, amount=i)))

print
results.sort(key=lambda x:x[1], reverse=True)
print results
print results[0]

10 20 30 40 50 60 70 80 90
[(17, 39), (18, 38), (16, 37), (20, 37), (21, 37), (39, 36), (44, 36), (52, 36), (13, 35), (19, 35), (23, 35), (37, 35), (40, 35), (45, 35), (46, 35), (47, 35), (51, 35), (53, 35), (57, 35), (58, 35), (59, 35), (60, 35), (15, 34), (36, 34), (38, 34), (41, 34), (42, 34), (43, 34), (48, 34), (49, 34), (50, 34), (54, 34), (56, 34), (61, 34), (62, 34), (14, 33), (22, 33), (24, 33), (25, 33), (30, 33), (31, 33), (34, 33), (35, 33), (55, 33), (63, 33), (64, 33), (65, 33), (81, 33), (82, 33), (84, 33), (85, 33), (89, 33), (93, 33), (32, 32), (66, 32), (67, 32), (68, 32), (69, 32), (71, 32), (80, 32), (83, 32), (86, 32), (87, 32), (88, 32), (90, 32), (91, 32), (92, 32), (94, 32), (95, 32), (96, 32), (97, 32), (98, 32), (99, 32), (33, 31), (70, 31), (72, 31), (73, 31), (74, 31), (75, 31), (76, 31), (77, 31), (78, 31), (79, 31), (11, 30), (27, 30), (29, 30), (12, 29), (26, 29), (9, 28), (28, 28), (7, 27), (8, 26), (10, 24), (6, 22), (5, 19)]
(17, 39)


In [30]:
#char, step = char3(15)

def mode5(char, step, amount):
    hist, char = more_hist(char, step, amount)
    train_mfcc, train_genre, test_mfcc, test_genre = train_test(char, hist)
    return classify2(train_mfcc, train_genre, test_mfcc, test_genre)

def classify2(train_mfcc, train_genre, test_mfcc, test_genre):
    success=0
    succ_gen = np.zeros(10)
    clf=SVC(kernel = 'rbf')
    clf.fit(train_mfcc,train_genre)
    for i in range(100):
        test=test_mfcc[i].reshape(1, -1)
        pred = clf.predict(test)
        if pred[0]==test_genre[i]:
            succ_gen[test_genre[i]] += 1
            success+=1
    print succ_gen, np.mean(succ_gen), success
    return success

for i in range(10,101):
    print str(i),":",
    mode5(char, (step*100)/i, amount=i)

 15
0: 0 10 20 30 40 50 60 70 80 90
1: 0 10 20 30 40 50 60 70 80 90
2: 0 10 20 30 40 50 60 70 80 90
3: 0 10 20 30 40 50 60 70 80 90
4: 0 10 20 30 40 50 60 70 80 90
5: 0 10 20 30 40 50 60 70 80 90
6: 0 10 20 30 40 50 60 70 80 90
7: 0 10 20 30 40 50 60 70 80 90
8: 0 10 20 30 40 50 60 70 80 90
9: 0 10 20 30 40 50 60 70 80 90
10 : [0. 9. 4. 0. 0. 1. 1. 8. 0. 1.] 2.4 24
11 : [6. 8. 1. 0. 0. 3. 0. 8. 1. 3.] 3.0 30
12 : [4. 4. 1. 1. 1. 5. 0. 8. 2. 4.] 3.0 30
13 : [4. 5. 0. 2. 1. 5. 3. 8. 2. 4.] 3.4 34
14 : [0. 6. 1. 1. 1. 4. 3. 8. 4. 5.] 3.3 33
15 : [0. 8. 1. 4. 1. 4. 3. 7. 6. 0.] 3.4 34
16 : [0. 8. 3. 5. 1. 4. 4. 8. 4. 0.] 3.7 37
17 : [0. 8. 3. 6. 1. 4. 5. 8. 4. 0.] 3.9 39
18 : [1. 8. 3. 3. 0. 4. 7. 9. 3. 0.] 3.8 38
19 : [ 1. 10.  3.  1.  0.  2.  7.  9.  2.  0.] 3.5 35
20 : [ 1. 10.  2.  2.  0.  1.  9.  9.  3.  0.] 3.7 37
21 : [ 2. 10.  2.  1.  0.  1.  9.  9.  4.  0.] 3.8 38
22 : [ 3. 10.  0.  1.  0.  0.  9.  9.  3.  0.] 3.5 35
23 : [3. 9. 0. 2. 0. 0. 9. 9. 2. 0.] 3.4 34
24 : [3. 9. 0. 1. 0.

KeyboardInterrupt: 

In [32]:
#leave one out
from sklearn.model_selection import LeaveOneOut
hist, char2 = more_hist(char, step, 90)

loo = LeaveOneOut()
loo.get_n_splits(char2)

for train_index, test_index in loo.split(char2):
    X_train, X_test = char2[train_index], char2[test_index]
    y_train, y_test = hist[train_index], hist[test_index]
    print(X_train, X_test, y_train, y_test)

 TRAIN: [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180
 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198
 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216
 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234
 235 236 237 238 239 240 241 242 243 244 24

NameError: name 'X' is not defined

# Results
* char = 5
* step = 18

* **result** = 36


* char = 20
* step = 18

* **result** = 37


* char = 12
* step = 18

* **result** = 37